# This code is based on Snake Coin code base

* Okay lets now try to create a blockchain that supports the following features
 * add transactions - sending money from one account to another account
 * mine - create blocks. Note this adds a transaction from network to miner - get some money
 * send blocks to others, receive blocks from others
 * achieve consensus on blocks
 
* This is the same block as before.
* we are going to work with JSON to transmit and receive blocks

In [ ]:


import hashlib as hasher
import json
import pprint
pp = pprint.PrettyPrinter(indent=4)
class Block:
  def __init__(self, index, timestamp, data, previous_hash):
    self.index = index
    self.timestamp = timestamp
    self.data = data

        
    self.previous_hash = previous_hash
    self.hash = self.hash_block()
  
  def hash_block(self):
    sha = hasher.sha256()
    sha.update(str(self.index).encode('utf-8') + 
               str(self.timestamp).encode('utf-8') + 
               str(self.data).encode('utf-8') + 
               str(self.previous_hash).encode('utf-8'))
  
    return sha.hexdigest()

* Now our genesis blocks contains the transactions in a list
* the proof of work is an entry that shows how we have finalized the block

In [1]:
import datetime as date

def create_genesis_block():
  # Manually construct a block with
  # index zero and arbitrary previous hash
  genesisdata = {
    "proof-of-work": 3,
    "transactions": []
  }
  return Block(0, date.datetime.now(), genesisdata, "0")

* Our blockchain is still a list

In [ ]:
# Create the blockchain and add the genesis block
blockchain = [create_genesis_block()]
previous_block = blockchain[0]

* We will use Flask to communicate with the blockchain.
* txion will add a JSON transaction


In [ ]:
from flask import Flask
from flask import request
node = Flask(__name__)

# Store the transactions that
# this node has in a list
this_nodes_transactions = []

@node.route('/txion', methods=['POST'])
def transaction():
  if request.method == 'POST':
    # On each new POST request,
    # we extract the transaction data
    new_txion = request.get_json()
    # Then we add the transaction to our list
    this_nodes_transactions.append(new_txion)
    # Because the transaction was successfully
    # submitted, we log it to our console
    print ("New transaction")
    print ("FROM: {}".format(new_txion['from']))
    print ("TO: {}".format(new_txion['to']))
    print ("AMOUNT: {}\n".format(new_txion['amount']))
    # Then we let the client know it worked out
    return "Transaction submission successful\n"


* proof of work will find the next number that is divisible by last proof of work and the number 9. Just an arbitrary computation to slow down the creation of blocks. Why do we need to do this?

* Note that before finalizing the block we add all received transactions
* we also add a transaction to send money to the miner from the network.

In [ ]:

miner_address = "q3nf394hjg-random-miner-address-34nf3i4nflkn3oi"

def proof_of_work(last_proof):
  # Create a variable that we will use to find
  # our next proof of work
  incrementor = last_proof + 1
  # Keep incrementing the incrementor until
  # it's equal to a number divisible by 9
  # and the proof of work of the previous
  # block in the chain
  while not (incrementor % 9 == 0 and incrementor % last_proof == 0):
    incrementor += 1
  # Once that number is found,
  # we can return it as a proof
  # of our work
  return incrementor

@node.route('/mine', methods = ['GET'])
def mine():
  # Get the last proof of work
  last_block = blockchain[len(blockchain) - 1]
  last_proof = last_block.data['proof-of-work']
  # Find the proof of work for
  # the current block being mined
  # Note: The program will hang here until a new
  #       proof of work is found
  proof = proof_of_work(last_proof)
  # Once we find a valid proof of work,
  # we know we can mine a block so 
  # we reward the miner by adding a transaction
  this_nodes_transactions.append(
    { "from": "network", "to": miner_address, "amount": 1 }
  )
  # Now we can gather the data needed
  # to create the new block
  new_block_data = {
    "proof-of-work": proof,
    "transactions": list(this_nodes_transactions)
  }
  new_block_index = last_block.index + 1
  new_block_timestamp = this_timestamp = date.datetime.now()
  last_block_hash = last_block.hash
  # Empty transaction list
  this_nodes_transactions[:] = []
  # Now create the
  # new block!
  mined_block = Block(
    new_block_index,
    new_block_timestamp,
    new_block_data,
    last_block_hash
  )
  blockchain.append(mined_block)
  # Let the client know we mined a block
  data= json.dumps({
      "index": new_block_index,
      "timestamp": str(new_block_timestamp),
      "data": new_block_data,
      "prvBlock": last_block_hash,
      "currentHash":mined_block.hash
  }) + "\n"
 
  pp.pprint(data)
  return data

* we send the blocks to others and receive it
* the consensus is the longest chain rule
* note that due to consensus mechanism it is possible that we will lose out some transactions.
* However, if the blockchain is stable around 10 new blocks have been added then the probability of losing out transactions is almost zero.

In [ ]:

@node.route('/blocks', methods=['GET'])
def get_blocks():
  chain_to_send = blockchain
  jsonversion=[]
  # Convert our blocks into dictionaries
  # so we can send them as json objects later
  for block in chain_to_send:
    block_index = str(block.index)
    block_timestamp = str(block.timestamp)
    block_data = str(block.data)
    block_hash = block.hash
    previoushash=block.previous_hash
    block = {
      "index": block_index,
      "timestamp": block_timestamp,
      "data": block_data,
      "hash": block_hash,
      "previous block": previous_hash 
    }
    jsonversion.append(block)
  # Send our chain to whomever requested it
  
  return json.dumps(jsonversion)

def find_new_chains():
  # Get the blockchains of every
  # other node
  other_chains = []
  for node_url in peer_nodes:
    # Get their chains using a GET request
    block = requests.get(node_url + "/blocks").content
    # Convert the JSON object to a Python dictionary
    block = json.loads(block)
    # Add it to our list
    other_chains.append(block)
  return other_chains

def consensus():
  # Get the blocks from other nodes
  other_chains = find_new_chains()
  # If our chain isn't longest,
  # then we store the longest chain
  longest_chain = blockchain
  for chain in other_chains:
    if len(longest_chain) < len(chain):
      longest_chain = chain
  # If the longest chain wasn't ours,
  # then we set our chain to the longest
  blockchain = longest_chain

In [ ]:
node.run()

```
curl "localhost:5000/txion"      -H "Content-Type: application/json"      -d '{"from": "fkjflw", "to":"fjlakdj", "amount": 4}'
  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100    81  100    34  100    47    155    214 --:--:-- --:--:-- --:--:--   369Transaction submission successful
```




```
curl localhost:5000/mine
```
```
  % Total    % Received % Xferd  Average Speed   Time    Time     Time  CurrentDload  Upload   Total   Spent    Left  Speed
100   358  100   358    0     0   1529      0 --:--:-- --:--:-- --:--:--  1529

{
"index": 4, 
"timestamp": "2018-11-07 22:28:34.985955",
"data": {"proof-of-work": 9, 
    "transactions": [{"from": "network", "to": "q3nf394hjg-random-miner-address-34nf3i4nflkn3oi", "amount": 1}]}, 
 "prvBlock": "be910c95d0e15ff380c748bf3b4e7062a6fb3c86cd55a7ade20c7002c91c3dc1", 
 "currentHash": "42dce73f8a56cfd01b91a2549816ef01423a6d5f00404064d3346c4ebe38fd03"
}
```

* try curl localhost:5000/blocks